In [16]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
import cv2
from unified_planning.shortcuts import *
import os
from anthropic import Anthropic
import base64
import ast
import itertools
import time



In [17]:
#Grid Sizes
x_size = 5
y_size = 5

In [18]:
#Open Image and Split into Grid

im = cv2.imread("legoscanbetter.jpg")

height, width = im.shape[:2]

grid_height = round(height / 5)
grid_width = round(width / 5)

sections = []
for y in range(y_size):
    for x in range(x_size):
        y_start = y * grid_height
        y_end = y_start + grid_height
        x_start = x * grid_width
        x_end = x_start + grid_width
        square = im[y_start:y_end, x_start:x_end]
        sections.append(square)
        cv2.imwrite(f"grid_{y}_{x}.png", square)  #

print(sections)


[array([[[ 33,  63,  80],
        [ 38,  68,  85],
        [ 46,  76,  93],
        ...,
        [134, 144, 144],
        [137, 147, 147],
        [138, 148, 148]],

       [[ 36,  66,  83],
        [ 42,  72,  89],
        [ 50,  80,  97],
        ...,
        [135, 145, 145],
        [137, 147, 147],
        [138, 148, 148]],

       [[ 36,  66,  83],
        [ 40,  70,  87],
        [ 48,  78,  95],
        ...,
        [136, 146, 146],
        [137, 147, 147],
        [137, 147, 147]],

       ...,

       [[ 65,  95, 112],
        [ 65,  95, 112],
        [ 63,  93, 110],
        ...,
        [155, 165, 165],
        [155, 165, 165],
        [156, 166, 166]],

       [[ 63,  93, 110],
        [ 64,  94, 111],
        [ 64,  94, 111],
        ...,
        [158, 168, 168],
        [162, 172, 172],
        [163, 173, 173]],

       [[ 60,  90, 107],
        [ 62,  92, 109],
        [ 64,  94, 111],
        ...,
        [161, 171, 171],
        [166, 176, 176],
        [166, 176, 176]

In [20]:
##Starting Prompt
#https://jdhao.github.io/2020/03/17/base64_opencv_pil_image_conversion/

count = 0
messages = []
replies = []
_, im_arr = cv2.imencode('.jpg', im)
im_bytes = im_arr.tobytes()
im_b64 = base64.b64encode(im_bytes).decode("utf-8")

messages.append({"role": "user",
                 "content": [

                     {
                         "type": "text",
                         "text": "I will start with providing you the overall image, then I will provide the same image in subsections so you can better identify elements. Can you identify all elements and give me them in the form [Unique Identifier, Label , Colour, Dimensions]? Every single unique identifier should be distinct from one another. After you have identified an element within a subsection confirm that it exists within the main image. You may identify an element which is positioned on the cross section of multiple subsections, identify this and assign the element to one of subsections. An element can appear multiple times, this is ok just ensure that the unique identifier differs for each occasion. Remember that the elements can be stacked on each other, so you may have to reason about the size of an element as some parts may be hidden by the element on top."
                     },
                     {"type": "image",
                      "source": {
                          "type": "base64",
                          "media_type": "image/jpeg",
                          "data": im_b64
                      }
                      }

                 ]})
print(len(sections))
for section in sections:
    time.sleep(5)
    count +=1
    _, im_arr = cv2.imencode('.png', section)
    im_bytes = im_arr.tobytes()
    im_b64 = base64.b64encode(im_bytes).decode("utf-8")
    messages.append(
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/png",
                        "data": im_b64
                    }
                }
            ]
        })

    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=1000,
        messages=messages
    )

    print(count)
    reply = response.content[0].text
    print(reply)
    replies.append(reply)
    messages.append({
        "role": "assistant",
        "content": reply
    })


25


BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'You have reached your specified API usage limits. You will regain access on 2025-09-01 at 00:00 UTC.'}}

In [7]:
print(replies)

['In the subsection image, I can identify the following element:\n\n[E1, Lego Brick, Brown, 2x3 studs]\n\nThis brown Lego brick with 2x3 studs can be found in the main image, located in the bottom left corner.', 'In this subsection image, I can identify the following elements:\n\n[E2, Lego Brick, Red, 2x2 studs]\n[E3, Lego Brick, Green, 2x4 studs]\n\nThe red 2x2 Lego brick (E2) and the green 2x4 Lego brick (E3) can both be found in the main image, located towards the center.', 'In this subsection image, I can identify the following elements:\n\n[E4, Lego Brick, Brown, 2x2 studs]\n[E5, Lego Brick, Tan, 2x2 studs]\n[E6, Lego Brick, White, 1x1 stud]\n\nThe brown 2x2 Lego brick (E4), the tan 2x2 Lego brick (E5), and the white 1x1 Lego brick (E6) can all be found in the main image, located towards the center.', 'In this subsection image, I can identify the following elements:\n\n[E7, Lego Brick, Red, 2x2 studs]\n[E8, Transparent Plastic Element, Clear, Rectangular]\n[E9, Eye Piece, Black]\n

In [18]:
count = 0
results = []
for repl in replies:
    temp = []
    for line in repl.splitlines():
        if line.startswith('['):
            line = line.replace('[', '')
            line = line.replace(']', '')
            arr = line.split(",")
            temp.append([count,arr[0],arr[1],arr[2],arr[3]])
    count +=1
    results.append(temp)
print(results)

[[[0, 'A1', ' Brick', ' Brown', ' 2x4']], [[1, 'B1', ' Brick', ' Red', ' 2x2'], [1, 'B2', ' Brick', ' Green', ' 2x4']], [[2, 'C1', ' Brick', ' Brown', ' 2x2'], [2, 'C2', ' Brick', ' Tan', ' 2x2'], [2, 'C3', ' Brick', ' White', ' 2x2']], [[3, 'D1', ' Brick', ' Red', ' 2x2'], [3, 'D2', ' Brick', ' White', ' 1x1'], [3, 'D3', ' Brick', ' Transparent', ' 1x1'], [3, 'D4', ' Brick', ' Transparent', ' 1x1']], [[4, 'E1', ' Brick', ' Brown', ' 2x2'], [4, 'E2', ' Brick', ' White', ' 1x1']], [[5, 'F1', ' Brick', ' Red', ' 2x2'], [5, 'F2', ' Brick', ' White', ' 1x1']], [[6, 'G1', ' Brick', ' Red', ' 2x2'], [6, 'G2', ' Brick', ' White', ' 1x4']], [[7, 'H1', ' Brick', ' White', ' 1x2'], [7, 'H2', ' Brick', ' White', ' 1x1']], [[8, 'I1', ' Brick', ' White', ' 1x1'], [8, 'I2', ' Brick', ' Black', ' 1x1']], [[9, 'J1', ' Brick', ' Red', ' 2x2'], [9, 'J2', ' Brick', ' Brown', ' 2x2']], [[10, 'K1', ' Brick', ' Red', ' 2x2'], [10, 'K2', ' Brick', ' White', ' 1x1']], [[11, 'L1', ' Brick', ' Transparent', ' 1

In [8]:
messages.append(
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "I will query you about multiple pairings of LEGO elements, you must tell me whether the relationship between them is on, left or behind. Reply in the form for example, UniqueIdentifier_A relationship UniqueIdentifier_B. On means if element A is on top of element B, left is if element A is to the left of element B, behind is if element A is behind element B. A relationship of on cannot occur with left or behind but an object could be left and behind."
                }
            ]
        })


for grid in results:
    print(grid)
    for g1, g2 in itertools.combinations(grid, 2):
        print(g1, g2)
        tester = f"What is the relationship between element {g1[1]} and element {g2[1]}?"
        messages.append(
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": tester
                }
            ]
        })

        positioning_response = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        messages=messages
        )

        reply = positioning_response.content[0].text
        print(reply)
        replies.append(reply)
        messages.append({
            "role": "assistant",
            "content": reply
        })



[[0, '1', ' Brick', ' Brown', ' 2x4']]
[[1, '2', ' Brick', ' Green', ' 2x4'], [1, '3', ' Brick', ' Red', ' 2x2']]
[1, '2', ' Brick', ' Green', ' 2x4'] [1, '3', ' Brick', ' Red', ' 2x2']
2 left 3
[[2, '1', ' Brick', ' Brown', ' 2x4'], [2, '4', ' Brick', ' Tan', ' 2x2'], [2, '5', ' Brick', ' White', ' 1x1']]
[2, '1', ' Brick', ' Brown', ' 2x4'] [2, '4', ' Brick', ' Tan', ' 2x2']
1 behind 4
[2, '1', ' Brick', ' Brown', ' 2x4'] [2, '5', ' Brick', ' White', ' 1x1']
1 left 5
[2, '4', ' Brick', ' Tan', ' 2x2'] [2, '5', ' Brick', ' White', ' 1x1']
4 on 5
[[3, '3', ' Brick', ' Red', ' 2x2'], [3, '6', ' Plate', ' Transparent', ' 1x2'], [3, '7', ' Plate', ' Transparent', ' 1x1'], [3, '8', ' Plate', ' White', ' 1x1'], [3, '9', ' Plate', ' White', ' 1x2']]
[3, '3', ' Brick', ' Red', ' 2x2'] [3, '6', ' Plate', ' Transparent', ' 1x2']
3 left 6
[3, '3', ' Brick', ' Red', ' 2x2'] [3, '7', ' Plate', ' Transparent', ' 1x1']
3 on 7
[3, '3', ' Brick', ' Red', ' 2x2'] [3, '8', ' Plate', ' White', ' 1x1']
3 

Sofishticated!
